# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [3819]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [3820]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3821]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [3822]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [3823]:
players_180_190 = len(player_data[(player_data["height"]<=190.0) & (player_data["height"]>=180.0)])

In [3824]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [3825]:
player_data['birthday'] = pd.to_datetime(player_data['birthday'])
players_1980 = len(player_data[(player_data["birthday"].dt.year == 1980)])

In [3826]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [3827]:
highest_players = player_data.sort_values(by=["weight", "player_name"], ascending=(False, True)).head(10)["player_name"].values.tolist()

In [3828]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [3829]:
ext_player_data = player_data[(player_data["birthday"].dt.year >= 1980) & (player_data["birthday"].dt.year <= 1990)]
ext_player_data["birthyear"] = player_data["birthday"].dt.year
years_born_players = [(idx,val) for idx, val in ext_player_data.groupby("birthyear")["birthyear"].size().iteritems()]

C:\Users\Raman_Susla\AppData\Local\Temp\ipykernel_22444\2465698832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ext_player_data["birthyear"] = player_data["birthday"].dt.year


In [3830]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [3831]:
player_data['first_name'] = player_data['player_name'].str.split(expand=True)[0]
adriano_height_data = player_data[(player_data["first_name"]=="Adriano")]["height"]
adriano_mean, adriano_std = adriano_height_data.mean(), adriano_height_data.std()

In [3832]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [3833]:
ext_player_data["dof"] = player_data["birthday"].dt.day_name()
dow_with_min_players_born = ext_player_data.groupby("dof")["dof"].size().idxmin()

C:\Users\Raman_Susla\AppData\Local\Temp\ipykernel_22444\3448144573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ext_player_data["dof"] = player_data["birthday"].dt.day_name()


In [3834]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [3835]:
match_data = pd.read_sql("SELECT * FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)
match_league_max = match_data.groupby("league_id")["league_id"].size()
match_league_max = match_league_max.where(match_league_max == match_league_max.max()).dropna()
league_most_matches = league_data.join(match_league_max, on="id", how="right")["name"].min()

In [3836]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [3837]:
from functools import reduce
match_data_on = []
for i in range(1, 12):
  match_data_on.append(f'home_player_{i}')
  match_data_on.append(f'away_player_{i}')

result_series = pd.Series()
for col in match_data_on:
  result_series=result_series.add(match_data.groupby(col)[col].size(), fill_value=0)

max_matches_player = player_data[player_data["player_api_id"]==int(result_series.idxmax())]["player_name"].iat[0]

C:\Users\Raman_Susla\AppData\Local\Temp\ipykernel_22444\2465658099.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result_series = pd.Series()


In [3838]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [3839]:
import numpy as np
player_attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
ext_player_attributes_data = player_attributes_data.loc[:, ~player_attributes_data.columns.isin(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])]
ext_player_attributes_data = ext_player_attributes_data.corr().abs()
ext_player_attributes_data = ext_player_attributes_data.where(np.triu(np.ones(ext_player_attributes_data.shape), k=1).astype(bool))
top_5_most_corr = ext_player_attributes_data.unstack().dropna().sort_values(ascending=False).head(5)
top_correlated_features = [correlation[0] for correlation in top_5_most_corr.iteritems()]

In [3840]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [3917]:
import math 
ext_player_attributes_data = player_attributes_data.merge(player_data, how="inner", left_on="player_api_id", right_on="player_api_id").sort_values("date", ascending=False).drop_duplicates(subset=["player_api_id"], keep='first')
neymar_p_join_pa = ext_player_attributes_data[ext_player_attributes_data["first_name"]=="Neymar"]
except_neymar_p_join_pa = pd.concat([ext_player_attributes_data, neymar_p_join_pa]).drop_duplicates(keep=False)

player_attribute_data_cols = player_attributes_data.loc[:, ~player_attributes_data.columns.isin(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])].columns.tolist()

neymar_p_join_pa = neymar_p_join_pa[player_attribute_data_cols].to_numpy()[0]

def euclidian_distance(row):
  row["result"] = np.sqrt(pow(row[player_attribute_data_cols].to_numpy() - neymar_p_join_pa, 2).sum())
  return row

neymar_similarities = except_neymar_p_join_pa.apply(euclidian_distance, axis=1).dropna(subset=["result"]).sort_values(by=["result", "player_name"]).head(5)["player_name"].to_list()

In [3842]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [3927]:
team_data = pd.read_sql("SELECT * FROM Team;", db)
team_league_matches = league_data[league_data["name"]=="Germany 1. Bundesliga"].merge(match_data[match_data["season"]=="2008/2009"].merge(team_data[team_data["team_long_name"]=="Borussia Dortmund"], left_on="home_team_api_id", right_on="team_api_id"), left_on="id", right_on="league_id")
borussia_bundesliga_2008_2009_matches = len(team_league_matches.index)

17


In [3844]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [3935]:
league_match = league_data[league_data["name"]=="Germany 1. Bundesliga"].merge(match_data[match_data["season"]=="2008/2009"], left_on="id", right_on="league_id")
home=league_match.merge(team_data, left_on="home_team_api_id", right_on="team_api_id").groupby("team_api_id").size()
away=league_match.merge(team_data, left_on="away_team_api_id", right_on="team_api_id").groupby("team_api_id").size()
team_most_matches_bundesliga_2008_2009 = (home+away).max().item() 

In [3846]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [3938]:
arsenal_home = match_data[match_data["season"]=="2015/2016"].merge(team_data[team_data["team_long_name"]=="Arsenal"], left_on="home_team_api_id", right_on="team_api_id")
arsenal_away = match_data[match_data["season"]=="2015/2016"].merge(team_data[team_data["team_long_name"]=="Arsenal"], left_on="away_team_api_id", right_on="team_api_id")
arsenal_won_matches_2015_2016 = len(arsenal_away[arsenal_away["away_team_goal"]>arsenal_away["home_team_goal"]].index) + len(arsenal_home[arsenal_home["home_team_goal"]>arsenal_home["away_team_goal"]].index)

20


In [3848]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [3947]:
home = match_data[match_data["season"]=="2015/2016"].merge(team_data, left_on="home_team_api_id", right_on="team_api_id")[["team_long_name", "home_team_goal", "away_team_goal"]]
away = match_data[match_data["season"]=="2015/2016"].merge(team_data, left_on="away_team_api_id", right_on="team_api_id")[["team_long_name", "home_team_goal", "away_team_goal"]]
total_home = home.groupby("team_long_name").size()
total_away = away.groupby("team_long_name").size()
won_home = home[home["home_team_goal"]>home["away_team_goal"]].groupby("team_long_name").size()
won_away = away[away["away_team_goal"]>away["home_team_goal"]].groupby("team_long_name").size()
won = won_home + won_away
total = total_home + total_away
team_highest_winrate_2015_2016 = (won/total).idxmax()

In [3850]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [3959]:
match_data['date'] = pd.to_datetime(match_data['date'])
season_all_sorted = league_data[league_data["name"]=="England Premier League"].merge(match_data[match_data["season"]=="2010/2011"], left_on="id", right_on="league_id").sort_values("date")
home=season_all_sorted.groupby("home_team_api_id")["home_team_api_id"].size()
away=season_all_sorted.groupby("away_team_api_id")["away_team_api_id"].size()
teams = (home+away).index.tolist()

highest_gap_england_2010_2011 = max([season_all_sorted[(season_all_sorted["home_team_api_id"]==team) | (season_all_sorted["away_team_api_id"]==team)]["date"].diff().max().days for team in teams])

In [3852]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [3853]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")